<h1 style='text-align:center; font-weight:bold'>Imports</h1>

In [57]:
import os
import gc
import cv2
import time
import pickle
import pandas as pd
from tqdm import tqdm
from rembg import remove

try:
    from kaggle.api.kaggle_api_extended import KaggleApi
except OSError:
    from kaggle.api.kaggle_api_extended import KaggleApi

<h1 style='text-align:center; font-weight:bold'>Downloading and Unpacking Data</h1>

In [58]:
def downloadData(path='./Dataset', name="fatihkgg/affectnet-yolo-format"):
    api = KaggleApi()
    if not os.path.exists('./Dataset'): os.makedirs(path)
    api.dataset_download_files(name, path=path, unzip=True)
    print("Dataset downloaded successfully")
    
# downloadData()

<h1 style='text-align:center; font-weight:bold'>Global Variables</h1>

In [59]:
LOAD_BATCH_SIZE = 500
CLASS_NAMES = [
    "Anger",
    "Contempt",
    "Disgust",
    "Fear",
    "Happy",
    "Neutral",
    "Sad",
    "Surprise",
]

<h1 style='text-align:center; font-weight:bold'>Preparing Data</h1>

In [60]:

def prepareData(type):
    imageDir = f"Dataset/YOLO_format/{type}/images"
    labelDir = f"Dataset/YOLO_format/{type}/labels"
    outputDir = f"Labeled Images/{type}"

    # Ensure output directories exist
    if not os.path.exists(outputDir):
        os.makedirs(outputDir)

    for className in CLASS_NAMES:
        classDir = os.path.join(outputDir, className)
        if not os.path.exists(classDir):
            os.makedirs(classDir)

    xData = []
    yData = []

    # Process each image
    for imageFile in os.listdir(imageDir):
        if not (imageFile.endswith(".jpg") or imageFile.endswith(".png")):
            continue

        imagePath = os.path.join(imageDir, imageFile)
        labelFile = imageFile.replace(".jpg", ".txt").replace(".png", ".txt")
        labelPath = os.path.join(labelDir, labelFile)

        if not os.path.exists(labelPath):
            continue

        image = cv2.imread(imagePath)
        if image is None:
            print(f"Warning: Unable to read image {imagePath}. Skipping...")
            continue

        with open(labelPath, "r") as f:
            for line in f.readlines():
                try:
                    data = line.strip().split()
                    classId = int(data[0])
                    className = CLASS_NAMES[classId]

                    # Save the image in the corresponding class directory
                    classDir = os.path.join(outputDir, className)
                    fileName = f"{os.path.splitext(imageFile)[0]}_{className}.{imageFile.split('.')[-1]}"
                    savePath = os.path.join(classDir, fileName)

                    # Save the image
                    cv2.imwrite(savePath, image)
                    print(f"Image saved at {savePath}")

                    # Optionally, you can append the image to xData and its corresponding label to yData
                    xData.append(image)
                    yData.append(className)

                except Exception as e:
                    print(f"Error processing line '{line}' in {labelPath}: {e}")

    return {"x": xData, "y": yData}



In [61]:
print(f"\n\t\t\t\t     Loading Data:")
print(f"\n********************************************************************************************\n")

xTrain, yTrain = [], []
try:
    batch = prepareData("train")  # Call prepareData and directly store the result in batch
    xTrain.extend(batch["x"])  # Extend with the "x" data (images)
    yTrain.extend(batch["y"])  # Extend with the "y" data (labels)
        
    print(f"Training: {len(xTrain)} images, {len(yTrain)} labels")
    print("Training Data is Successfully Saved")
    
except Exception as e:
    print(f"Error Loading Training Data: {e}")

time.sleep(1)
try:
    del xTrain, yTrain
except Exception as e:
    print(f"Error Deleting Training Data: {e}")
        
print(f"\n********************************************************************************************\n")
gc.collect()

xVal, yVal = [], []
try:
    batch = prepareData("valid")  # Call prepareData and directly store the result in batch
    xVal.extend(batch["x"])  # Extend with the "x" data (images)
    yVal.extend(batch["y"])  # Extend with the "y" data (labels)
        
    print(f"Validation: {len(xVal)} images, {len(yVal)} labels")
    print(f"Validation Data is Successfully Saved")
    
except Exception as e:
    print(f"Error Loading Validation Data: {e}") 

time.sleep(1)
try:
    del xVal, yVal
except Exception as e:
    print(f"Error Deleting Validation Data: {e}")
        
print(f"\n********************************************************************************************\n")
gc.collect()

xTest, yTest = [], []
try:
    batch = prepareData("test")  
    xTest.extend(batch["x"])  # Extend with the "x" data (images)
    yTest.extend(batch["y"])  # Extend with the "y" data (labels)
        
    print(f"Testing: {len(xTest)} images, {len(yTest)} labels")
    print("Testing Data is Successfully Saved")
    
except Exception as e:
    print(f"Error Loading Testing Data: {e}")   
        
time.sleep(1)
try:
    del xTest, yTest
except Exception as e:
    print(f"Error Deleting Testing Data: {e}")


				     Loading Data:

********************************************************************************************

Image saved at Labeled Images/train\Happy\ffhq_0_Happy.png
Image saved at Labeled Images/train\Happy\ffhq_1_Happy.png
Image saved at Labeled Images/train\Disgust\ffhq_10_Disgust.png
Image saved at Labeled Images/train\Surprise\ffhq_100_Surprise.png
Image saved at Labeled Images/train\Contempt\ffhq_1000_Contempt.png
Image saved at Labeled Images/train\Happy\ffhq_1001_Happy.png
Image saved at Labeled Images/train\Happy\ffhq_1003_Happy.png
Image saved at Labeled Images/train\Neutral\ffhq_1004_Neutral.png
Image saved at Labeled Images/train\Anger\ffhq_1005_Anger.png
Image saved at Labeled Images/train\Happy\ffhq_1007_Happy.png
Image saved at Labeled Images/train\Neutral\ffhq_1008_Neutral.png
Image saved at Labeled Images/train\Contempt\ffhq_1009_Contempt.png
Image saved at Labeled Images/train\Surprise\ffhq_101_Surprise.png
Image saved at Labeled Images/train\Happy\ffhq_10